In [1]:
import re
from unidecode import unidecode
from string import punctuation

# Define text preprocessing function
def preprocess_text(text):
    # Remove punctuation and normalize text
    text = unidecode(text.lower())
    text = re.sub(f"[{punctuation}]", "", text)
    return text

# Example
input_text = "Hello! What's your name?"
clean_text = preprocess_text(input_text)
print("Cleaned Text:", clean_text)


Cleaned Text: hello whats your name


In [2]:
# Symbol-to-ID mapping (simplified)
_symbol_to_id = {s: i for i, s in enumerate("abcdefghijklmnopqrstuvwxyz ", 1)}

# Convert text to sequence of IDs
def text_to_sequence(text):
    sequence = [_symbol_to_id.get(s, 0) for s in text if s in _symbol_to_id]
    return sequence

# Example usage
sequence = text_to_sequence(clean_text)
print("Text to Sequence:", sequence)

Text to Sequence: [8, 5, 12, 12, 15, 27, 23, 8, 1, 20, 19, 27, 25, 15, 21, 18, 27, 14, 1, 13, 5]


In [ ]:
import re
from unidecode import unidecode
from string import punctuation

# Define text preprocessing function
def preprocess_text(text):
    # Remove punctuation and normalize text
    text = unidecode(text.lower())
    text = re.sub(f"[{punctuation}]", "", text)
    return text

# Example
input_text = "Hello! What's your name?"
clean_text = preprocess_text(input_text)
print("Cleaned Text:", clean_text)

import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tacotron2 = torch.hub.load("nvidia/DeepLearningExamples:torchhub", "nvidia_tacotron2").to(device).eval()

# Function to generate mel spectrogram from text
def generate_mel_spectrogram(text):
    # Convert text to sequence tensor
    sequence = torch.tensor(text_to_sequence(text), dtype=torch.long).unsqueeze(0).to(device)
    input_lengths = torch.tensor([sequence.shape[1]], dtype=torch.long).to(device)

    with torch.no_grad():
        mel_spec, _, _ = tacotron2.infer(sequence, input_lengths)
    return mel_spec

# Generate mel spectrogram
mel_spec = generate_mel_spectrogram(clean_text)
print("Mel Spectrogram Shape:", mel_spec.shape)

In [ ]:
import re
import torch
import IPython.display as ipd
from unidecode import unidecode
from string import punctuation

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Tacotron 2 and WaveGlow models from PyTorch Hub and set them to evaluation mode
tacotron2 = torch.hub.load("nvidia/DeepLearningExamples:torchhub", "nvidia_tacotron2").to(device).eval()
waveglow = torch.hub.load("nvidia/DeepLearningExamples:torchhub", "nvidia_waveglow").to(device).eval()
waveglow = waveglow.remove_weightnorm(waveglow)  # Remove weight normalization for faster inference

# Symbol to ID mapping for Tacotron2 text processing
_symbol_to_id = {s: i for i, s in enumerate("what is your name", 1)}

# Function to clean and convert text to sequence
def text_to_sequence(text):
    text = unidecode(text.lower())
    text = re.sub(f"[{punctuation}]", "", text)
    sequence = [_symbol_to_id.get(s, 0) for s in text if s in _symbol_to_id]
    return torch.tensor(sequence, dtype=torch.long).unsqueeze(0).to(device)

# Define the TTS function
def tts(text):
    # Convert text to sequence and get input length
    sequence = text_to_sequence(text)
    input_lengths = torch.tensor([sequence.shape[1]], dtype=torch.long).to(device)
    
    # Generate mel spectrogram using Tacotron 2
    with torch.no_grad():
        # Updated line to get the first element of the tuple which is the mel spectrogram.
        mel_spec, _, _ = tacotron2.infer(sequence, input_lengths)  
    
    # Generate audio waveform from mel spectrogram using WaveGlow
    with torch.no_grad():
        audio = waveglow.infer(mel_spec)
    
    return audio.cpu().numpy()

# Example text
text = "Hello, this is a demonstration of text-to-speech synthesis using Tacotron 2 and WaveGlow."

# Generate audio waveform from text
audio_waveform = tts(text)

# Play audio
ipd.Audio(audio_waveform, rate=22050)  # Sample rate for WaveGlow

In [ ]:
second example

In [3]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
inputs = processor(text="Don't count the days, make the days count.", return_tensors="pt")
from datasets import load_dataset
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
import torch
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)
spectrogram = model.generate_speech(inputs["input_ids"], speaker_embeddings)
from transformers import SpeechT5HifiGan
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)
from IPython.display import Audio
Audio(speech, rate=16000)

RuntimeError: Failed to import transformers.models.speecht5.processing_speecht5 because of the following error (look up to see its traceback):
operator torchvision::nms does not exist

In [ ]:
Step 1: Dataset Preparation (Example with LJSpeech)
To load the dataset, we use torchaudio for audio files and prepare input text as a sequence of phonemes or characters.
import torch
from torch.utils.data import Dataset, DataLoader
import torchaudio
from torchaudio.datasets import LJSPEECH
import torchaudio.transforms as transforms

class TTSDataset(Dataset):
    def __init__(self, root, transform=None):
        self.dataset = LJSPEECH(root, download=True)
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        waveform, sample_rate, transcript, *_ = self.dataset[idx]
        
        # Convert text to a sequence of phonemes or characters (basic example)
        text_sequence = torch.tensor([ord(c) for c in transcript.lower() if c.isalnum()], dtype=torch.long)
        
        # Generate mel spectrogram
        mel_spectrogram = transforms.MelSpectrogram()(waveform)
        
        return text_sequence, mel_spectrogram.squeeze(0)
    
# Load dataset
dataset = TTSDataset(root="./data")
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=lambda x: x)

Step 2: Define a Simplified TTS Model
For demonstration purposes, we’ll define a basic encoder-decoder model that generates a mel spectrogram from text input. In a real TTS system, this would be more complex (e.g., Tacotron 2 with attention mechanisms).
import torch.nn as nn
class SimpleTTSModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=256, hidden_dim=512, mel_bins=80):
        super(SimpleTTSModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.decoder = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, mel_bins)

    def forward(self, text_sequence):
        embedded = self.embedding(text_sequence)
        encoder_output, _ = self.encoder(embedded)
        decoder_output, _ = self.decoder(encoder_output)
        mel_spectrogram = self.fc(decoder_output)
        return mel_spectrogram


Step 3: Define Loss Functions
The model uses a spectrogram loss (e.g., mean squared error) to compare the generated and target mel spectrograms. Optionally, you can add a stop token loss for more complex TTS systems.
import torch.nn.functional as F
def spectrogram_loss(pred, target):
    return F.mse_loss(pred, target)
def stop_token_loss(pred_stop, target_stop):
    return F.binary_cross_entropy_with_logits(pred_stop, target_stop)
Step 4: Training Loop
Now we’ll set up the training loop, including the optimizer and gradient clipping.
# Model parameters
vocab_size = 128  # ASCII characters used in the text sequence
model = SimpleTTSModel(vocab_size).to("cuda" if torch.cuda.is_available() else "cpu")

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training loop
epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for epoch in range(epochs):
    model.train()
    total_loss = 0.0
   
    for text_sequence, target_mel in dataloader:
        # Move to device
        text_sequence = text_sequence.to(device)
        target_mel = target_mel.to(device)
        # Forward pass
        predicted_mel = model(text_sequence)
        
        # Calculate loss
        loss = spectrogram_loss(predicted_mel, target_mel)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
        optimizer.step()
        
        total_loss += loss.item()
    
    # Print average loss for this epoch
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")
